In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('anime.csv')

In [3]:
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
df.sample(10)

,anime_id,name,genre,type,episodes,rating,members
5490,5132,Let&#039;s Nupu Nupu,"Comedy, Ecchi, School",TV,16,6.44,1861
3513,882,Yuu☆Yuu☆Hakusho (Movie),"Action, Adventure, Comedy",Movie,1,6.97,13904
74,21,One Piece,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",TV,Unknown,8.58,504862
3339,1192,Kakurenbo,"Horror, Psychological, Supernatural",Movie,1,7.02,22853
9653,26103,Namae... Sore wa Moeru Inochi,"Drama, Kids",OVA,1,8.33,50
9650,32776,Nagisa,Music,Music,1,3.85,69
9942,32143,Peeping Life x Kaijuu Sakaba Kaiji: Kaijuu-tac...,"Comedy, Slice of Life, Supernatural",OVA,10,4.00,93
9005,32252,Japonese,Comedy,Movie,1,6.67,35
9238,6012,Kinpatsu no Jeanie,"Drama, Historical",TV,13,6.53,275
2119,16468,One Piece: Glorious Island,"Action, Adventure, Comedy, Fantasy, Shounen, S...",Special,2,7.35,12922


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [6]:
df.shape

(12294, 7)

In [7]:
df.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


Selecting just ONA and Special for  movie type

In [8]:
other_anime = df[(df['type']=='ONA')|(df['type']=="Special")]

In [9]:
other_anime.shape

(2335, 7)

In [10]:
other_anime.describe()

,anime_id,rating,members
count,2335.000000,2323.000000,2335.000000
mean,18533.829979,6.276453,6670.759743
std,11119.395599,1.038972,14811.019789
min,191.000000,1.670000,24.000000
25%,8244.500000,5.690000,197.000000
50%,18795.000000,6.430000,1292.000000
75%,30048.000000,7.010000,5949.500000
max,34519.000000,8.660000,160423.000000


In [11]:
percent_tile = other_anime['members'].quantile(0.75)

In [12]:
percent_tile

5949.5

In [13]:
other_anime = other_anime[(other_anime['members']>= percent_tile)]

In [14]:
other_anime.shape

(584, 7)

Selecting just TV and Movie for movie type

In [15]:
anime = df[(df['type']=='TV')|(df['type']=="Movie")]

In [16]:
anime.describe()

,anime_id,rating,members
count,6135.000000,5968.000000,6.135000e+03
mean,13088.271557,6.677433,3.031281e+04
std,11388.556425,1.051031,7.425729e+04
min,1.000000,1.920000,5.000000e+00
25%,2809.500000,6.150000,2.290000e+02
50%,9253.000000,6.800000,1.984000e+03
75%,23023.000000,7.390000,2.304150e+04
max,34527.000000,10.000000,1.013917e+06


In [17]:
new_anime = anime['members'].quantile(0.75)

In [18]:
new_anime

23041.5

In [19]:
new_anime = anime[(anime['members']>= new_anime)]

In [20]:
new_anime.shape

(1534, 7)

### ADD RATINGS DATASET

In [21]:
ratings=  pd.read_csv('rating_anime.csv')

In [22]:
ratings

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [23]:
ratings.sample()

,user_id,anime_id,rating
6262450,58410,1034,6


In [24]:
ratings.loc[ratings.rating ==-1,'rating']=np.NaN

In [25]:
ratings.sample(10)

,user_id,anime_id,rating
7009543,65497,12291,9.0
6354538,58984,355,NaN
7575058,70970,8841,6.0
6166824,57694,9253,8.0
1130629,10837,3712,NaN
5002218,47872,24,10.0
2075417,20100,17513,8.0
371140,3788,10456,9.0
337029,3436,16498,8.0
1620315,15739,20853,NaN


In [26]:
ratings.shape

(7813737, 3)

#### Create index for anime name

indexing for  ONA and Special for movie type

In [27]:
other_index = pd.Series(other_anime.index,index= other_anime.name)

In [28]:
other_index

name
Mushishi Special: Hihamukage                48
Mushishi Zoku Shou: Odoro no Michi          85
Steins;Gate: Oukoubakko no Poriomania      126
Hajime no Ippo: Champion Road              162
Gintama: Shiroyasha Koutan                 171
                                         ...  
Tenshi no Drop                            8013
Shitcom                                   8052
Bible Black: Imari Ryoujoku Genba        11393
Manyuu Hikenchou Specials                11702
Seikon no Qwaser II Picture Drama        11732
Length: 584, dtype: int64

indexing for TV and Movie for movie type

In [29]:
anime_index = pd.Series(df.index,index= df.name)

In [30]:
anime_index.head()

name
Kimi no Na wa.                      0
Fullmetal Alchemist: Brotherhood    1
Gintama°                            2
Steins;Gate                         3
Gintama&#039;                       4
dtype: int64

In [31]:
anime_index.sample(10)

name
Akatsuki no Yona OVA                                                  642
Tantei Kageki Milky Holmes TD                                        5404
Mahou Shoujo Pretty Sammy                                            5251
Futari Ecchi                                                         5097
Arei no Kagami: Way to the Virgin Space                              6417
Match Uri no Shoujo                                                  6215
D.C.if: Da Capo if                                                   2326
Hand Soap                                                            7717
New Initial D Movie: Legend 2 - Tousou                                681
Gakuen no Ikenie: Nagusami Mono to Kashita Kyonyuu Furyou Shoujo    11992
dtype: int64

# Join the data

joining data for ONA and Special for movie type

In [32]:
other_joined = other_anime.merge(ratings, how ="inner", on = 'anime_id')

In [33]:
other_joined.head(5)

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,21329,Mushishi Special: Hihamukage,"Adventure, Fantasy, Historical, Mystery, Seine...",Special,1,8.66,49036,39,NaN
1,21329,Mushishi Special: Hihamukage,"Adventure, Fantasy, Historical, Mystery, Seine...",Special,1,8.66,49036,43,8.0
2,21329,Mushishi Special: Hihamukage,"Adventure, Fantasy, Historical, Mystery, Seine...",Special,1,8.66,49036,46,9.0
3,21329,Mushishi Special: Hihamukage,"Adventure, Fantasy, Historical, Mystery, Seine...",Special,1,8.66,49036,54,NaN
4,21329,Mushishi Special: Hihamukage,"Adventure, Fantasy, Historical, Mystery, Seine...",Special,1,8.66,49036,73,8.0


In [34]:
other_joined.shape

(582172, 9)

joining data for TV and Movie for Movie type

In [35]:
joined = new_anime.merge(ratings, how ="inner", on = 'anime_id')

In [36]:
joined.head(5)

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,5.0
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,10.0
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,10.0
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,10.0
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,278,NaN


In [37]:
joined.shape

(5863210, 9)

### create a pivot table

create a pivotal table  for ONA and Special for movie type

In [38]:
other_joined = other_joined[['user_id', 'name', 'rating_y']]

In [39]:
other_pivot = pd.pivot_table(other_joined, index='name', columns='user_id', values='rating_y')

In [40]:
other_pivot.shape

(581, 42748)

create a pivotal table  for TV and Movie for movie type

In [41]:
other_pivot.sample(4)

user_id,3,5,7,11,12,14,17,19,20,21,...,73499,73500,73502,73503,73504,73507,73508,73510,73512,73515
name,,,,,,,,,,,,,,,,,,,,,
Toshokan Sensou: Koi no Shougai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Ano Natsu de Matteru: Bokutachi wa Koukou Saigo no Natsu wo Sugoshinagara, Ano Natsu de Matteiru.",NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Loveless Specials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kyou no Asuka Show,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
joined = joined[['user_id', 'name', 'rating_y']]

In [43]:
pivot = pd.pivot_table(joined, index='name', columns='user_id', values='rating_y')

In [44]:
pivot.sample(10)

user_id,1,2,3,5,7,8,9,10,11,12,...,73507,73508,73509,73510,73511,73512,73513,73514,73515,73516
name,,,,,,,,,,,,,,,,,,,,,
Sankarea,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Petshop of Horrors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
White Album 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nobunaga the Fool,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Detective Conan Movie 02: The Fourteenth Target,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lance N&#039; Masques,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jigoku Shoujo Futakomori,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genei wo Kakeru Taiyou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Komori-san wa Kotowarenai!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
pivot.shape

(1490, 69224)

### Drop all the users thay never rate an anime

droping NaN for pivotal table  for ONA and Special for movie type

In [46]:
other_pivot.dropna(axis=1, how='all', inplace =True)
other_pivot.head()

user_id,3,5,7,11,12,14,17,19,20,21,...,73499,73500,73502,73503,73504,73507,73508,73510,73512,73515
name,,,,,,,,,,,,,,,,,,,,,
.hack//G.U. Trilogy: Parody Mode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Intermezzo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Unison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11eyes: Momoiro Genmutan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN


In [47]:
other_pivot.shape

(581, 42748)

droping NaN for pivotal table  for TV and Movie for movie type

In [48]:
pivot.dropna(axis=1, how='all', inplace =True)
pivot.head()

user_id,1,2,3,5,7,8,9,10,11,12,...,73507,73508,73509,73510,73511,73512,73513,73514,73515,73516
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Roots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Sign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Tasogare no Udewa Densetsu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-Ghost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
pivot.shape

(1490, 69224)

### CAL MEAN AND COMPRESS EVERY USER

normalizing  for ONA and Special for movie type

In [50]:
other_pivot_norm = other_pivot.apply(lambda x: x - np.nanmean(x), axis=1)
other_pivot_norm.head()

user_id,3,5,7,11,12,14,17,19,20,21,...,73499,73500,73502,73503,73504,73507,73508,73510,73512,73515
name,,,,,,,,,,,,,,,,,,,,,
.hack//G.U. Trilogy: Parody Mode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Intermezzo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Unison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11eyes: Momoiro Genmutan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.596531,NaN,...,NaN,NaN,NaN,NaN,NaN,-3.403469,NaN,NaN,NaN,NaN


normalizing for ONA and Special for movie type

In [51]:
pivot_norm = pivot.apply(lambda x: x - np.nanmean(x), axis=1)
pivot_norm.head()

user_id,1,2,3,5,7,8,9,10,11,12,...,73507,73508,73509,73510,73511,73512,73513,73514,73515,73516
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Roots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Sign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Tasogare no Udewa Densetsu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-Ghost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Item based collaborative filtering

Filling the NaN with 0 for ONA and Special for movie type

In [52]:
#Fill NaN with 0
other_pivot_norm.fillna(0, inplace = True)
other_pivot_norm.head()

user_id,3,5,7,11,12,14,17,19,20,21,...,73499,73500,73502,73503,73504,73507,73508,73510,73512,73515
name,,,,,,,,,,,,,,,,,,,,,
.hack//G.U. Trilogy: Parody Mode,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
.hack//Intermezzo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
.hack//Unison,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
11eyes: Momoiro Genmutan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.596531,0.0,...,0.0,0.0,0.0,0.0,0.0,-3.403469,0.0,0.0,0.0,0.0


Filling the NaN with 0 for  TV and Movie for movie type

In [53]:
#Fill NaN with 0
pivot_norm.fillna(0, inplace = True)
pivot_norm.head()

user_id,1,2,3,5,7,8,9,10,11,12,...,73507,73508,73509,73510,73511,73512,73513,73514,73515,73516
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Movie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Roots,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07-Ghost,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Calculate SImilar items

similarity difference for ONA and Special for movie type

In [54]:
#convert into dataframe to make it easier
other_tem_sim_df = pd.DataFrame(cosine_similarity(other_pivot_norm, other_pivot_norm), index=other_pivot_norm.index, columns=other_pivot_norm.index)

In [55]:
other_tem_sim_df.head()

name,.hack//G.U. Trilogy: Parody Mode,.hack//Intermezzo,.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,.hack//Unison,11eyes: Momoiro Genmutan,91 Days Recap,A-Channel: +A-Channel,Aa! Megami-sama! (TV) Specials,Aa! Megami-sama!: Sorezore no Tsubasa Specials,Aa! Megami-sama!: Tatakau Tsubasa,...,"Yuru Yuri: Doushite, Tomaranai, Tokimeki, Dokidoki, Paradox, Eternal",Yuyushiki: Nyanyashiki,Yu☆Gi☆Oh! Duel Monsters: Battle City Special,Yu☆Gi☆Oh!: Chou Yuugou! Toki wo Koeta Kizuna,Zero no Tsukaima: Princesses no Rondo - Yuuwaku no Sunahama,Zero no Tsukaima: Princesses no Rondo Picture Drama,Zombie-Loan Specials,ef: A Tale of Melodies. - Prologue,ef: A Tale of Memories. - Prologue,ef: A Tale of Memories. - Recollections
name,,,,,,,,,,,,,,,,,,,,,
.hack//G.U. Trilogy: Parody Mode,1.000000,0.223721,0.227872,0.220550,0.035053,0.013149,0.034011,0.027011,0.008713,0.018634,...,0.029036,-0.001424,0.031739,0.008052,0.038360,0.042970,0.044298,0.025894,0.025427,0.034907
.hack//Intermezzo,0.223721,1.000000,0.249006,0.665174,0.047134,0.011671,0.028913,0.041120,0.047661,0.041544,...,0.017600,0.002533,0.026293,0.009074,0.053378,0.019020,0.042872,0.031901,0.041238,0.028955
.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,0.227872,0.249006,1.000000,0.258469,0.036312,0.016527,0.044404,0.056334,0.017775,0.019630,...,0.027519,-0.000451,0.022852,0.004873,0.033165,0.033158,0.034430,0.028442,0.020475,0.033077
.hack//Unison,0.220550,0.665174,0.258469,1.000000,0.028002,0.009374,0.023931,0.039831,0.049317,0.042134,...,0.006604,0.003389,0.030826,0.012495,0.044502,0.018100,0.046095,0.042112,0.040929,0.038484
11eyes: Momoiro Genmutan,0.035053,0.047134,0.036312,0.028002,1.000000,0.018501,0.071524,0.060274,0.033141,0.034249,...,0.040747,0.011252,0.036067,0.058380,0.098970,0.062523,0.048575,0.039526,0.056112,0.039252


similarity difference for TV and Movie for movie type

In [56]:
#convert into dataframe to make it easier
item_sim_df = pd.DataFrame(cosine_similarity(pivot_norm, pivot_norm), index=pivot_norm.index, columns=pivot_norm.index)

In [57]:
item_sim_df.head()

name,&quot;Bungaku Shoujo&quot; Movie,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,07-Ghost,11eyes,30-sai no Hoken Taiiku,91 Days,A-Channel,AKB0048,...,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,ef: A Tale of Melodies.,ef: A Tale of Memories.,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Movie,1.000000,0.032753,0.026011,0.036676,0.038012,0.046374,0.047874,0.009425,0.056224,0.040200,...,0.048061,0.036628,0.045698,0.031965,0.073987,0.079183,0.008384,0.035791,0.049389,0.047023
.hack//Roots,0.032753,1.000000,0.289985,0.315053,0.074391,0.061119,0.027616,0.006698,0.027652,0.031912,...,0.043802,0.034236,0.066003,0.004396,0.038025,0.030098,0.049731,0.038284,0.040426,0.031665
.hack//Sign,0.026011,0.289985,1.000000,0.269825,0.058850,0.050163,0.033374,0.014148,0.035317,0.031722,...,0.038653,0.033260,0.056693,0.005344,0.049085,0.050667,0.067435,0.055709,0.031301,0.033710
.hack//Tasogare no Udewa Densetsu,0.036676,0.315053,0.269825,1.000000,0.047875,0.053741,0.027466,0.009595,0.040570,0.023247,...,0.038657,0.034183,0.063091,0.000434,0.044590,0.036658,0.072843,0.048902,0.041610,0.038522
07-Ghost,0.038012,0.074391,0.058850,0.047875,1.000000,0.122489,0.040180,0.021060,0.051344,0.035019,...,0.094213,0.017888,0.129566,0.007980,0.042737,0.036837,0.041351,0.054222,0.066368,0.042567


functionas to get similarity difference for ONA and Special for movie type

In [58]:
def get_similar_anime_for_others(anime_name):
    if anime_name not in other_pivot_norm.index:
        return None, None
    else:
        other_sim_animes = other_tem_sim_df.sort_values(by=anime_name, ascending=False).index[1:]
        other_sim_score = other_tem_sim_df.sort_values(by=anime_name, ascending=False).loc[:, anime_name].tolist()[1:]
        return other_sim_animes, other_sim_score

In [59]:
other_animes, other_score = get_similar_anime_for_others(".hack//Intermezzo")
for x, y in zip(other_animes[:10], other_score[:10]):
    print("{} with similarity of {}".format(x, y))

.hack//Unison with similarity of 0.6651739572284352
.hack//Tasogare no Udewa Densetsu: Offline de Aimashou with similarity of 0.24900649334290625
.hack//G.U. Trilogy: Parody Mode with similarity of 0.22372069385494966
Full Metal Panic! The Second Raid: Wari to Hima na Sentaichou no Ichinichi with similarity of 0.07172152324076231
Druaga no Tou: The Aegis of Uruk - Jil no Bouken with similarity of 0.06843810165007623
Petit Eva: Evangelion@School with similarity of 0.06700921153247016
Kurokami: Tora to Tsubasa with similarity of 0.06661951366184804
Shakugan no Shana Specials with similarity of 0.0640383167867054
Air Gear Special with similarity of 0.0640374031216414
Darker than Black: Kuro no Keiyakusha Special with similarity of 0.06244851434544533


functions to predict ratings for ONA and Special for movie type

In [60]:
#predict the rating of anime x by user y
def predict_rating(user_id, anime_name, max_neighbor=10):
    other_animes, other_scores = get_similar_anime_for_others(anime_name)
    other_anime_arr = np.array([x for x in other_animes])
    other_sim_arr = np.array([x for x in other_scores])
    
    #select only the anime that has already rated by user x
    other_filtering = other_pivot_norm[user_id].loc[other_anime_arr] != 0
    
    #calculate the predicted score
    other_s = np.dot(other_sim_arr[other_filtering][:max_neighbor], other_pivot[user_id].loc[other_anime_arr[other_filtering][:max_neighbor]]) / np.sum(other_sim_arr[other_filtering][:max_neighbor])
    
    return other_s

In [61]:
predict_rating(5, '11eyes: Momoiro Genmutan')

2.7310405852909097

functions to get similarity difference for TV and Movie for movie type

In [62]:
def get_similar_anime(anime_name):
    if anime_name not in pivot_norm.index:
        return None, None
    else:
        sim_animes = item_sim_df.sort_values(by=anime_name, ascending=False).index[1:]
        sim_score = item_sim_df.sort_values(by=anime_name, ascending=False).loc[:, anime_name].tolist()[1:]
        return sim_animes, sim_score

In [63]:
animes, score = get_similar_anime(".hack//Unison")
for x, y in zip(animes[:10], score[:10]):
    print("{} with similarity of {}".format(x, y))

TypeError: 'NoneType' object is not subscriptable

functions to predict ratings for TV and Movie for movie type

In [ ]:
#predict the rating of anime x by user y
def predict_rating(user_id, anime_name, max_neighbor=10):
    animes, scores = get_similar_anime(anime_name)
    anime_arr = np.array([x for x in animes])
    sim_arr = np.array([x for x in scores])
    
    #select only the anime that has already rated by user x
    filtering = pivot_norm[user_id].loc[anime_arr] != 0
    
    #calculate the predicted score
    s = np.dot(sim_arr[filtering][:max_neighbor], pivot[user_id].loc[anime_arr[filtering][:max_neighbor]]) / np.sum(sim_arr[filtering][:max_neighbor])
    
    return s

In [ ]:
predict_rating(5, '11eyes: Momoiro Genmutan')